### Ingest drivers.json file

In [0]:
# Create a text widget for data source input with an empty default value
dbutils.widgets.text("p_data_source", "")

# Retrieve the value of the data source widget
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
# Create a text widget for file date input with a default value of "2021-03-21"
dbutils.widgets.text("p_file_date", "2021-03-21")

# Retrieve the value of the file date widget
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
# Running the configuration notebook to load necessary configurations and settings
%run "../includes/configuration"

In [0]:
# Run the common functions notebook to make its functions available in the current notebook
%run "../includes/common_functions"

##### Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
# Import necessary classes from pyspark.sql.types to define schema for DataFrames
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:
# Define schema for the 'name' field in the drivers DataFrame
name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
])

In [0]:
# Define schema for the drivers DataFrame
drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [0]:
# Read the JSON file into a DataFrame using the defined schema
drivers_df = spark.read \
    .schema(drivers_schema) \
    .json(f"{raw_folder_path}/{v_file_date}/drivers.json")

##### Step 2 - Rename columns and add new columns
1. driverId renamed to driver_id  
1. driverRef renamed to driver_ref  
1. ingestion date added
1. name added with concatenation of forename and surname

In [0]:
# Import specific functions from pyspark.sql.functions module
from pyspark.sql.functions import col, concat, lit

In [0]:
# Add an ingestion date column to the DataFrame
drivers_with_ingestion_date_df = add_ingestion_date(drivers_df)

In [0]:
# Rename columns and add new columns to the DataFrame
drivers_with_columns_df = drivers_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) \
                                    .withColumn("data_source", lit(v_data_source)) \
                                    .withColumn("file_date", lit(v_file_date))

##### Step 3 - Drop the unwanted columns
1. name.forename
1. name.surname
1. url

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

##### Step 4 - Write to output to processed container in parquet format

In [0]:
# Write the DataFrame to a Delta table in overwrite mode
drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.drivers")

In [0]:
%sql
-- Select all columns from the drivers table in the f1_processed schema
SELECT * FROM f1_processed.drivers

In [0]:
# Exit the notebook with a success message
dbutils.notebook.exit("Success")